In [3]:
import sys

!{sys.executable} -m pip install numpy==1.19.2 --user
!{sys.executable} -m pip install scipy==1.6.2 --user
!{sys.executable} -m pip install ta --user
!{sys.executable} -m pip install datetime --user
!{sys.executable} -m pip install yfinance --user
!{sys.executable} -m pip install MetaTrader5 --user

     |████████████████████████████████| 15.3 MB 23.6 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28986 sha256=19fa3fed51f7d63585b496cb389f80668b180259336e597d8daea515771e391c
  Stored in directory: /Users/juangabriel/Library/Caches/pip/wheels/18/9a/81/694fa8602da445fa009fd13c8da25001be19efdfb67a9cc348
Successfully built ta
     |████████████████████████████████| 51 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 2.0 MB/s eta 0:00:011
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8488 sha256=464b015e696019c4bbeb2915e0281594b248900ca08545b6be77a7fe15d58b1d
  Stored in directory: /Users/juangabriel/Library/Caches/pip/wheels/21/c9/66/b41c847de65c7985db52ec21d59996841598b8b0e93f2b9500
Successfully built multitasking
  Consider adding this directory to PATH or, if you prefer to su

In [12]:
import ta
import numpy as np
import warnings
from datetime import datetime
import pandas as pd
import MetaTrader5 as mt5
warnings.filterwarnings("ignore")
mt5.initialize()
from Chapter_09_MT5 import *
import time





def SMA_strategy(symbol):
    
    # Import / Features engineering
    df = MT5.get_data(symbol, 70)
    
    # Creamos la Resistencia haciendo un rolling máximo
    df["SMA fast"] = df["close"].rolling(30).mean()

    # Creamos el Soporte haciendo un rolling mínimo
    df["SMA slow"] = df["close"].rolling(60).mean()


    # Señales
    condition_1_buy = df["SMA fast"].iloc[-1] > df["SMA slow"].iloc[-1]
    
    condition_1_sell = df["SMA fast"].iloc[-1] < df["SMA slow"].iloc[-1]
    
    buy = condition_1_buy
    sell = condition_1_sell
    

    return buy, sell


mt5.initialize()
# True = Live Trading and False = Screener
live = True

if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")


info_order = {
    "Bitcoin": ["EURUSD", 1.]
    
}


start = datetime.now().strftime("%H:%M:%S") #"23:59:59"
while True:

    # Verificación para lanzar el algoritmo
    if datetime.now().weekday() not in (5,6): # Para hacer trading solamente de Lunes a Viernes
        is_time = datetime.now().strftime("%H:%M:%S") == start 
    else:
        is_time = False

    
    # Lanzamos el algoritmo
    if is_time:

        # Abrimos el trading
        for asset in info_order.keys():

            # Inicializamos la entrada de datos
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Creamos la señal
            buy, sell = SMA_strategy(symbol)

             # Ejecutamos el algoritmo
            if live:
                MT5.run(symbol, buy, sell,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")


------------------------------------------------------------------
Date:  2022-01-14 08:33:32
Balance: 10000.0 USD, 	Equity: 9991.28 USD, 	Profit: -8.72 USD
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2022-01-14 08:33:32
SYMBOL: BTCUSD
BUY: False 	  SHORT: False
POSITION: None 	 ID: None
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2022-01-14 08:33:32
SYMBOL: XMRUSD
BUY: False 	  SHORT: False
POSITION: None 	 ID: None
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2022-01-14 08:33:32
SYMBOL: ETHUSD
BUY: False 	  SHORT: False
POSITION: None 	 ID: None
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2022-01-14 08:3

KeyboardInterrupt: 